In [ ]:
import pandas as pd

import statsmodels.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

%matplotlib inline
import seaborn as sns
import glob, os

# ANOVA

In [ ]:
keyfile="/path/to/key.tsv"
key=pd.read_csv(keyfile, 
                delim_whitespace=True)
def anova(file):
    df=pd.read_csv(file, 
                delim_whitespace=True,
                names=["salinity", "temperature", "num", "id"])
    df=df[~df.id.str.contains("kcont")]
    for index, row in key.iterrows():
        label=str(row[3]) + "_" +row[4]
        if label not in df['id'].values:
            df=df.append({'salinity':row[2],'temperature':row[0],'num':0.0,'id':label}, ignore_index=True)
    X = (df[['salinity','temperature']]).values
    y = (df['num']).values
    formula = 'num ~ C(salinity) + C(temperature) + C(salinity):C(temperature)'
    model = ols(formula, df).fit()
    aov_table = anova_lm(model, typ=2)
    sp=aov_table['PR(>F)']['C(salinity)']
    tp=aov_table['PR(>F)']['C(temperature)']
    stp=aov_table['PR(>F)']['C(salinity):C(temperature)']
    return [sp,tp,stp]

In [ ]:
spvals=[]
tpvals=[]
stpvals=[]
for file in os.listdir("/path/to/input_prot/"):
    protein=file[0:-4]
    pvalue=anova(os.path.join("/path/to/input_prot/", file))
    sp=pvalue[0]
    tp=pvalue[1]
    stp=pvalue[2]
    spvals.append([protein,sp])
    tpvals.append([protein,tp])
    stpvals.append([protein,stp])

In [ ]:
#Steps explicitly written out for variable clarity
#Salinity
spin=list(map(list, zip(*spvals)))[1]
snan=[spin for spin in spin if str(spin) == 'nan']
spin=[spin for spin in spin if str(spin) != 'nan']
sort_spin= sorted(spin, key=float)

#Temperature
tpin=list(map(list, zip(*tpvals)))[1]
tnan=[spin for spin in spin if str(spin) == 'nan']
tpin=[tpin for tpin in tpin if str(tpin) != 'nan']
sort_tpin= sorted(tpin, key=float)

#Salinity & Temperature
stpin=list(map(list, zip(*stpvals)))[1]
stnan=[spin for spin in spin if str(spin) == 'nan']
stpin=[stpin for stpin in stpin if str(stpin) != 'nan']
sort_stpin= sorted(stpin, key=float)

# Benjamini-Hochberg

In [ ]:
npa = np.asarray(spin, dtype=np.float32)
#Salinity
sq=statsmodels.stats.multitest.multipletests(npa)
sq=statsmodels.stats.multitest.multipletests(npa, method="fdr_bh")
npa = np.asarray(tpin, dtype=np.float32)
#Temperature
tq=statsmodels.stats.multitest.multipletests(npa)
tq=statsmodels.stats.multitest.multipletests(npa, method="fdr_bh")
npa = np.asarray(stpin, dtype=np.float32)
#Salinity & Temperature
stq=statsmodels.stats.multitest.multipletests(npa)
stq=statsmodels.stats.multitest.multipletests(npa, method="fdr_bh")
#s=statsmodels.stats.multitest.multipletests(pin, alpha=0.05, method='fdr_bh', is_sorted=False, returnsorted=False)

# Get proteins from Benjamini-Hochberg step

In [ ]:
def bhp(qvals, pvals, num):
    bhprot=[]
    proteins=list(map(list, zip(*pvals)))[0]
    p=0
    for i in range(0,len(qvals[0])):
        while np.isnan(pvals[p][1]):
            p+=1
        keep=qvals[0][i]
        q=qvals[1][i]
        if keep:
            bhprot.append(proteins[p])
        p+=1
    return bhprot
    
s=len(spvals)-1
t=len(tpvals)-1
st=len(spvals)-1

sprot=bhp(sq, spvals, s)
tprot=bhp(tq, tpvals, t)
stprot=bhp(stq, stpvals, st)

# Fold Change

### Salinity

In [ ]:
i=0
onlys=[]
onlyb=[]
sbig=[]
bbig=[]
columns=['protein','Salinity fold change (log2(brine) - log2(seawater))','s','snum','b','bnum','q-value','-log(q-value)']
spdf=pd.DataFrame(columns=columns)
for p in sprot:
    for file in os.listdir("/path/to/input_prot/"):
        protein=file[0:-4]
        if protein == p:
            s=0
            snum=0
            b=0
            bnum=0
            f=os.path.join("/path/to/input_prot/", file)
            fdf=pd.read_csv(f, delim_whitespace=True,names=["salinity", "temperature", "num", "id"])
            fdf=fdf[~fdf.id.str.contains("kcont")]
            for index, row in fdf.iterrows():
                if row[0] == 's':
                    s+=1
                    snum+=row[2]/23
                elif row[0] == 'b':
                    b+=1
                    bnum+=row[2]/16
            if snum > bnum:
                sbig.append(p)
            elif bnum > snum:
                bbig.append(p)
            if bnum == 0:
                onlys.append(p)
            elif snum == 0:
                onlyb.append(p)
            #else:
            fs=np.log2(bnum)-np.log2(snum)
            invq=0-np.log(sq[1][i])
            spdf = spdf.append({'-log(q-value)':invq,'q-value':sq[1][i],'protein':p,
                          's':s,'snum':snum,
                          'b':b,'bnum':bnum,'Salinity fold change (log2(brine) - log2(seawater))':fs}, ignore_index=True)
    i+=1



### Temperature

In [ ]:
i=0
columns=['protein','Temperature fold change (log2(-10C) - log2(-5C))','t5','t5num','t10','t10num','q-value','-log(q-value)']
tpdf=pd.DataFrame(columns=columns)
only5=[]
only10=[]
t5big=[]
t10big=[]
for p in tprot:
    for file in os.listdir("/path/to/input_prot/"):
        protein=file[0:-4]
        if protein == p:
            t5=0
            t5num=0
            t10=0
            t10num=0
            f=os.path.join("/path/to/input_prot/", file)
            fdf=pd.read_csv(f, delim_whitespace=True,names=["salinity", "temperature", "num", "id"])
            fdf=fdf[~fdf.id.str.contains("kcont")]
            for index, row in fdf.iterrows():
                if row[1] == 5:
                    t5+=1
                    t5num+=row[2]/24
                elif row[1] == 10:
                    t10+=1
                    t10num+=row[2]/15
            if t5num > t10num:
                t5big.append(p)
            elif t10num > t5num:
                t10big.append(p)
            if t5num == 0:
                only10.append(p)
            elif t10num == 0:
                only5.append(p)
            #else:
            ft=np.log2(t10num)-np.log2(t5num)
            invq=0-np.log(tq[1][i])
            tpdf = tpdf.append({'-log(q-value)':invq,'q-value':tq[1][i],'protein':p,
                          't5':t5,'t5num':t5num,
                          't10':t10,'t10num':t10num, 'Temperature fold change (log2(-10C) - log2(-5C))':ft}, ignore_index=True)
    i+=1

### Salinity & Temperature

In [ ]:
not10=[]
not5=[]
nots=[]
notb=[]
isw=[]
ib=[]
i5=[]
i10=[]
i=0
columns=['protein','Temperature fold change (log2(-10C) - log2(-5C))','Salinity fold change (log2(brine) - log2(seawater))','s','snum','b','bnum','t5','t5num','t10','t10num','q-value','-log(q-value)']
stpdf=pd.DataFrame(columns=columns)
for p in stprot:
    for file in os.listdir("/path/to/input_prot/"):
        protein=file[0:-4]
        if protein == p:
            s=0
            snum=0
            b=0
            bnum=0
            t5=0
            t5num=0
            t10=0
            t10num=0
            f=os.path.join("/path/to/input_prot/", file)
            fdf=pd.read_csv(f, delim_whitespace=True,names=["salinity", "temperature", "num", "id"])
            fdf=fdf[~fdf.id.str.contains("kcont")]
            for index, row in fdf.iterrows():
                if row[0] == 's':
                    s+=1
                    snum+=row[2]/23
                elif row[0] == 'b':
                    b+=1
                    bnum+=row[2]/16
                if row[1] == 5:
                    t5+=1
                    t5num+=row[2]/24
                elif row[1] == 10:
                    t10+=1
                    t10num+=row[2]/15
            if t5num > t10num:
                i5.append(p)
            elif t10num > t5num:
                i10.append(p)
            if snum > bnum:
                isw.append(p)
            elif bnum > snum:
                ib.append(p)
            if t5num == 0:
                not5.append(p)
                if snum ==0:
                    nots.append(p)
                elif bnum ==0:
                    notb.append(p)
            elif t10num == 0:
                not10.append(p)
                if snum ==0:
                    nots.append(p)
                elif bnum ==0:
                    notb.append(p)
            else:
                if snum ==0:
                    nots.append(p)
                elif bnum ==0:
                    notb.append(p)
                #else:
            fs=np.log2(bnum+np.finfo(float).tiny)-np.log2(snum+np.finfo(float).tiny)
            ft=np.log2(t10num+np.finfo(float).tiny)-np.log2(t5num+.0000001)
            invq=1-stq[1][i]
            stpdf = stpdf.append({'-log(q-value)':invq,'q-value':stq[1][i],'protein':p,
                              'Temperature fold change (log2(-10C) - log2(-5C))':ft,'Salinity fold change (log2(brine) - log2(seawater))':fs,
                              's':s,'snum':snum,
                              'b':b,'bnum':bnum,
                              't5':t5,'t5num':t5num,
                              't10':t10,'t10num':t10num}, ignore_index=True)
    i+=1

# dbpdf

In [ ]:
i=0
columns=['protein','Temperature fold change (log2(-10C) - log2(-5C))','Salinity fold change (log2(brine) - log2(seawater))','s','snum','b','bnum','t5','t5num','t10','t10num','q-value','-log(q-value)']
dbpdf=pd.DataFrame(columns=columns)

for file in os.listdir("/path/to/input_prot/"):
    protein=file[0:-4]
    p=protein
    s=0
    snum=0
    b=0
    bnum=0
    t5=0
    t5num=0
    t10=0
    t10num=0
    s5num, s10num, b5num, b10num=0,0,0,0
    s5count, s10count, b5count, b10count=0,0,0,0
    f=os.path.join("/path/to/input_prot/", file)
    fdf=pd.read_csv(f, delim_whitespace=True,names=["salinity", "temperature", "num", "id"])
    fdf=fdf[~fdf.id.str.contains("kcont")]
    for index, row in fdf.iterrows():
        if row[0] == 's':
            s+=1
            snum+=row[2]/23
            if row[1] == 5:
                s5num+=row[2]/16
                s5count+=1/16
            elif row[1] == 10:
                s10num+=row[2]/7
                s10count+=1/7
        elif row[0] == 'b':
            b+=1
            bnum+=row[2]/16
            if row[1] == 5:
                b5num+=row[2]/8
                b5count+=1/8
            elif row[1] == 10:
                b10num+=row[2]/8
                b10count+=1/8
        if row[1] == 5:
            t5+=1
            t5num+=row[2]/24
        elif row[1] == 10:
            t10+=1
            t10num+=row[2]/15
    fs=np.log2(bnum+np.finfo(float).tiny)-np.log2(snum+np.finfo(float).tiny)
    ft=np.log2(t10num+np.finfo(float).tiny)-np.log2(t5num+.0000001)
    invq='nan'
    dbpdf = dbpdf.append({'-log(q-value)':invq,'q-value':stq[1][i],'protein':p,
                      'Temperature fold change (log2(-10C) - log2(-5C))':ft,'Salinity fold change (log2(brine) - log2(seawater))':fs,
                      's':s,'snum':snum,
                      'b':b,'bnum':bnum,
                      't5':t5,'t5num':t5num,
                      't10':t10,'t10num':t10num,
                      's5num':s5num,'s10num':s10num,
                      'b5num':b5num,'b10num':b10num,
                      's5count':s5count,'s10count':s10count,
                      'b5count':b5count,'b10count':b10count}, ignore_index=True)
i+=1

# sample matrix

In [ ]:

i=0
columns=['protein','Temperature fold change (log2(-10C) - log2(-5C))','Salinity fold change (log2(brine) - log2(seawater))','s','snum','b','bnum','t5','t5num','t10','t10num','q-value','-log(q-value)']
columns=list(map(list, zip(*spvals)))[0]
samplematrix={"conditions":{}}
for file in os.listdir("/path/to/input_prot/"):
    protein=file[0:-4]
    if protein in columns:
        f=os.path.join("/path/to/input_prot/", file)
        fdf=pd.read_csv(f, delim_whitespace=True,names=["salinity", "temperature", "num", "id"])
        fdf=fdf[~fdf.id.str.contains("kcont")]
        for index, row in fdf.iterrows():
            if protein not in samplematrix:
                samplematrix[protein]={}
            samplematrix[protein][row[3]]=row[2]
            tag=str(row[0])+str(row[1])
            samplematrix["conditions"][row[3]]=tag

In [ ]:
samplematrix

# T-test

In [ ]:
key=pd.read_csv("/path/to/key.tsv", 
                delim_whitespace=True)
def ttest(file, temp):
    df=pd.read_csv(file, 
                delim_whitespace=True,
                names=["salinity", "temperature", "num", "id"])
    df=df[~df.id.str.contains("kcont")]
    for index, row in key.iterrows():
        label=str(row[3]) + "_" +row[4]
        if label not in df['id'].values:
            df=df.append({'salinity':row[2],'temperature':row[0],'num':0.0,'id':label}, ignore_index=True)
    df=df[df["temperature"] == temp]
    ts=df[df["salinity"] == "s"][["num"]].values
    tb=df[df["salinity"] == "b"][["num"]].values
    ret=statsmodels.stats.weightstats.ttest_ind(ts,tb)
    return ret


In [ ]:
def runt(big, temp):
    out=[]
    for p in big:
        filename="/path/to/input_prot/"+p+".txt"
        pvalue=ttest(filename, temp)
        out.append([p,pvalue])
    return out




In [ ]:
salinity=sbig+bbig
t5t=runt(salinity,5)
t10t=runt(salinity,10)

In [ ]:
t5t[0][1][1][0]
t5l=len(t5t)-1
t5p=[]
pt5=0
pt10=0
for i in range(0,t5l):
    t5p.append([t5t[i][0],t5t[i][1][1][0]])
    if t5t[i][1][1][0] <.01:
        pt5+=1

t10l=len(t10t)-1
t10p=[]
for i in range(0,t10l):
    t10p.append([t10t[i][0],t10t[i][1][1][0]])
    if t10t[i][1][1][0] <.01:
        pt10+=1

In [ ]:
t5pin=list(map(list, zip(*t5p)))[1]
t5nan=[t5pin for t5pin in t5pin if str(t5pin) == 'nan']
t5pin=[t5pin for t5pin in t5pin if str(t5pin) != 'nan']

npa = np.asarray(t5pin, dtype=np.float32)
t5q=statsmodels.stats.multitest.multipletests(npa)
t5q=statsmodels.stats.multitest.multipletests(npa, method="fdr_bh")
l=len(t5p)-1
t5prot=bhp(t5q, t5p, l)


t10pin=list(map(list, zip(*t10p)))[1]
t10nan=[t10pin for t10pin in t10pin if str(t10pin) == 'nan']
t10pin=[t10pin for t10pin in t10pin if str(t10pin) != 'nan']
npa = np.asarray(t10pin, dtype=np.float32)
t10q=statsmodels.stats.multitest.multipletests(npa)
t10q=statsmodels.stats.multitest.multipletests(npa, method="fdr_bh")
l=len(t10p)-1
t10prot=bhp(t10q, t10p, l)

In [ ]:
whole=sbig+bbig
negative=set(sprot).intersection(set(tprot))
salinitynot10= list(set(whole).difference(set(negative)))+t5prot

In [ ]:
def split(sprot,temp,df):
    i=0
    scount='s'+str(temp)+'count'
    bcount='b'+str(temp)+'count'
    snum='s'+str(temp)+'num'
    bnum='b'+str(temp)+'num'
    allproteins=[]
    sbig=[]
    bbig=[]
    for index,row in df.iterrows():
        if str(row.protein) in sprot:
            if row[snum] > row[bnum]:
                sbig.append(row.protein)
            elif row[bnum] > row[snum]:
                bbig.append(row.protein)
        allproteins.append(row.protein)
    notsbig= list(set(allproteins)-set(sbig))
    notbbig= list(set(allproteins)-set(bbig))

    return sbig, bbig, notsbig, notbbig

s5,b5,nots5,notb5=split(t5prot,5,dbpdf)
#s5,b5,nots5,notb5=split(salinitynot10,5,dbpdf)
s10,b10,nots10,notb10=split(t10prot,10,dbpdf)


# Make output FASTA's

In [ ]:

proteindbf='/path/to/Colwellia_psychrerythraea_UniProt4.2019.contams.fasta'
#pdb=open(proteindbf,'r')
tempfasta=[]
def makefasta(dbf, proteins, out, pdf, column):
    fout=open(out, 'w')
    with open(dbf) as fp:
        for cnt, line in enumerate(fp):
            #print(line)
            if line[0] == '>':
                write=False
                name=line.split(" ")[0][1:]
                if name in proteins:
                    write=True
                    #print(name)
                    #print(column)
                    a=float(pdf[pdf['protein']==name][column])
                    #fout.write('\n>'+str(a)+" "+line[1:]) #FIX add mean abundance to weight the motifs
                    fout.write('\n>'+line[1:])
            else:
                if write:
                    fout.write(line.rstrip())
                    #fout.write(line)
                    #tempfasta.append(line)
    fout.close()



In [ ]:
out='/path/to/output/pfastas/'
myall=out+'all.txt'
proteindbf='/path/to/Colwellia_psychrerythraea_UniProt4.2019.contams.fasta'
fout=open(myall, 'w')
with open(proteindbf) as fp:
    for cnt, line in enumerate(fp):
        if line[0] == '>':
            name=line.split('|')[2]
            name=name.split(' ')[0]
            print(name)
            fout.write(name+'\n')
fout.close()


In [ ]:
#NOTall5,all10,seawater5,brine5,seawater10,brine10
dbbig=list(map(list, zip(*spvals)))[0]
ns5out=out+'not_seawater5.fa'
ns10out=out+'not_seawater10.fa'
nb5out=out+'not_brine5.fa'
nb10out=out+'not_brine10.fa'
nall5=out+'not_all5.fa'
nall10=out+'not_all10.fa'
nott5big=set(dbbig)-set(t5big)
nott10big=set(dbbig)-set(t10big)

proteindbf='/path/to/Colwellia_psychrerythraea_UniProt4.2019.contams.fasta'
#pdb=open(proteindbf,'r')
tempfasta=[]
def makelist(dbf, proteins, out, pdf, column):
    fout=open(out, 'w')
    for p in proteins:
        if len(p.split('|'))>2:
            name=p.split('|')[1]
            #print(name)
            fout.write(name+'\n')
    fout.close()
#all5,all10,seawater5,brine5,seawater10,brine10
out='/path/to/output/pfastas/final/lists/'
s5out=out+'seawater5.txt'
s10out=out+'seawater10.txt'
ns5out=out+'notseawater5.txt'
nb5out=out+'notbrine5.txt'
b5out=out+'brine5.txt'
b10out=out+'brine10.txt'
all5=out+'all5.txt'
all10=out+'all10.txt'


makelist(proteindbf,s5, s5out, dbpdf, 's5num') #seawater fasta
makelist(proteindbf,nots5, ns5out, dbpdf, 's5num') #nots5
makelist(proteindbf,notb5, nb5out, dbpdf, 'b5num') #notb5
makelist(proteindbf,s10,s10out, dbpdf, 's10num') #brine fasta
makelist(proteindbf,b5, b5out, dbpdf, 'b5num') #seawater fasta
makelist(proteindbf,b10,b10out, dbpdf, 's10num') #brine fasta
makelist(proteindbf,t5big,all5, tpdf, 't5num') #-5C fasta
makelist(proteindbf,t10big,all10, tpdf, 't10num') #-10C fasta
makelist(proteindbf,dbbig,myall, dbpdf, 't10num') #-10C fasta


In [ ]:



makefasta(proteindbf,nots5, ns5out, dbpdf, 's5num') #seawater fasta
makefasta(proteindbf,nots10,ns10out, dbpdf, 's10num') #brine fasta
makefasta(proteindbf,notb5, nb5out, dbpdf, 'b5num') #seawater fasta
makefasta(proteindbf,notb10,nb10out, dbpdf, 's10num') #brine fasta
makefasta(proteindbf,nott5big,nall5, dbpdf, 't5num') #-5C fasta
makefasta(proteindbf,nott10big,nall10, dbpdf, 't10num') #-10C fasta

# Heatmap

In [ ]:
#by mean instance counts all ANOVA out DICTIONARY
y=[]
z=[]
hdf=pd.DataFrame.from_dict(samplematrix).fillna(0)
proteins=list(set(sprot+tprot+stprot))
proteins.append("conditions")
hdf=hdf[proteins]

conditions=pd.Series(hdf.conditions)
lut = dict(zip(conditions.unique(), ["#D81B60","#1E88E5","#FFC107","#004D40"]))
row_colors = conditions.map(lut)
proteins=list(set(sprot+tprot+stprot))
hdf=hdf[proteins]
print(len(proteins))
sns.clustermap(hdf, row_colors=row_colors).fig.suptitle("Proteins from ANOVA by Sample")


In [ ]:
#by mean instance counts all T-Test out DICTIONARY
y=[]
z=[]
#x=["protein","Seawater & -5C","Seawater & -10C","Brine & -5C","Brine & -10C"]
hdf=pd.DataFrame.from_dict(samplematrix).fillna(0)
proteins=list(set(t5prot+t10prot))
proteins.append("conditions")
hdf=hdf[proteins]

conditions=pd.Series(hdf.conditions)
lut = dict(zip(conditions.unique(), ["#D81B60","#1E88E5","#FFC107","#004D40"]))
row_colors = conditions.map(lut)
proteins=list(set(t5prot+t10prot))
hdf=hdf[proteins]
print(len(proteins))
sns.clustermap(hdf, row_colors=row_colors).fig.suptitle("Proteins from ANOVA & T-test by Sample")